# Experiment 1

In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch import nn, optim

from plant_village_dataset import PlantVillageDataset
from runner import Runner
from resnet50 import ResNet50

## Prepare Data

Here we define the split according to project specifications, with the decision made to interpret 10% of the dataset to be used for training as including both the training and validation data, and the 10% for testing to be used wholly for testing proper.

In [ ]:
BATCH_SIZE = 128

In [ ]:
def split(dataset, batch_size, labeled_ratio, test_ratio):    
    labels = np.array([label for _, label in dataset])

    unlabeled_indices, labeled_indices = train_test_split(np.arange(len(dataset)),
                                                          test_size=labeled_ratio,
                                                          stratify=labels)   
    
    relative_test_ratio = test_ratio / labeled_ratio
    
    train_val_indices, test_indices = train_test_split(labeled_indices,
                                                       test_size=relative_test_ratio,
                                                       stratify=labels[labeled_indices])
    
    train_indices, val_indices = train_test_split(train_val_indices,
                                                  test_size=0.2,
                                                  stratify=labels[train_val_indices])

    unlabeled_sampler = SubsetRandomSampler(unlabeled_indices)
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    unlabeled_loader = DataLoader(dataset, batch_size=batch_size, sampler=unlabeled_sampler)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

    return unlabeled_loader, train_loader, val_loader, test_loader

In [ ]:
dataset = PlantVillageDataset('images')

## Run 1

In [ ]:
unlabeled_loader, train_loader, val_loader, test_loader = split(dataset, batch_size=BATCH_SIZE, labeled_ratio=0.2, test_ratio=0.1)

##### CNN

In [ ]:
cnn = ResNet50(num_classes=len(dataset.classes))
cnn_optim = optim.Adam(cnn.parameters(), lr=1e-3)
cnn_criterion = nn.CrossEntropyLoss()
cnn_runner = Runner('cnn_1', cnn, cnn_optim, cnn_criterion, device='mps')
cnn_runner.train(train_loader, val_loader, num_epochs=3)
cnn_runner.test(test_loader)
pass

##### Autoencoder

In [ ]:
# Declare UNetAutoEncoder
# Train UNetAutoEncoder
# Extract Encoder from UNetAutoEncoder

##### Frozen Encoder + MLP

In [ ]:
# Train one Frankenstein with the Encoder's weights set to eval() (Frozen)

##### Live Encoder + MLP

In [ ]:
# Train the second Frankenstein normally

## Run 2

In [ ]:
unlabeled_loader, train_loader, val_loader, test_loader = split(dataset, batch_size=BATCH_SIZE, labeled_ratio=0.5, test_ratio=0.15)

##### CNN

In [ ]:
cnn = ResNet50(num_classes=len(dataset.classes))
cnn_optim = optim.Adam(cnn.parameters(), lr=1e-3)
cnn_criterion = nn.CrossEntropyLoss()
cnn_runner = Runner('cnn_2', cnn, cnn_optim, cnn_criterion, device='mps')
cnn_runner.train(train_loader, val_loader, num_epochs=3)
cnn_runner.test(test_loader)
pass

##### Autoencoder

In [ ]:
# Declare UNetAutoEncoder
# Train UNetAutoEncoder
# Extract Encoder from UNetAutoEncoder

##### Frozen Encoder + MLP

In [ ]:
# Train one Frankenstein with the Encoder's weights set to eval() (Frozen)

##### Live Encoder + MLP

In [ ]:
# Train the second Frankenstein normally